# Text2Everything SDK - Complete End-to-End Example

This notebook demonstrates a complete workflow using the Text2Everything SDK:
**Project → Context → Golden Example → Schema → Connector → Session → Chat → Feedback**

## Prerequisites

1. Install the SDK: `pip install text2everything-sdk`
2. Have your API endpoint and API key ready
3. Ensure you have access to the Text2Everything API
4. Have a database connection available for the connector example

## Setup and Configuration

In [3]:
# Import the SDK
from text2everything_sdk import Text2EverythingClient
from text2everything_sdk.exceptions import (
    AuthenticationError,
    ValidationError,
    NotFoundError,
    RateLimitError
)
import os

print("✅ SDK imported successfully!")

# Configuration
BASE_URL = os.getenv("T2E_BASE_URL")  # Replace with your API endpoint
API_KEY = os.getenv("T2E_API_KEY")  # Replace with your API key

print(f"Connecting to: {BASE_URL}")

✅ SDK imported successfully!
Connecting to: http://localhost


## Initialize the Client

In [5]:
# Initialize the Text2Everything client
client = Text2EverythingClient(
    base_url=BASE_URL,
    api_key=API_KEY,
    timeout=60,
    max_retries=1,
    retry_delay=1.0
)

print("✅ Client initialized successfully!")
print(f"Base URL: {client.base_url}")

✅ Client initialized successfully!
Base URL: http://localhost


## 1. 🆕 Projects - Simplified Creation

In [ ]:
# 🆕 NEW: Direct keyword arguments
project = client.projects.create(
    name="E-commerce Analytics",
    description="Complete e-commerce data analysis project"
)

print("✅ Project created !")
print(f"Project ID: {project.id}")
print(f"Project Name: {project.name}")

PROJECT_ID = project.id

✅ Project created with new API pattern!
Project ID: a70a686a-d2cc-4755-a502-4772521041fc
Project Name: E-commerce Analytics


## 2. 🆕 Contexts - Simplified Creation

In [ ]:
# 🆕 NEW: Direct keyword arguments
context = client.contexts.create(
    project_id=PROJECT_ID,
    name="E-commerce Business Rules",
    description="Business rules and definitions for e-commerce analytics",
    content="""Business Rules:
- Revenue = quantity * unit_price
- Active customers have made a purchase in the last 90 days
- High-value customers have lifetime value > $1000

Definitions:
- LTV: Customer Lifetime Value
- AOV: Average Order Value
- Churn: Customers who haven't purchased in 180+ days
""",
    is_always_displayed=True
)

print("✅ Context created !")
print(f"Context ID: {context.id}")
print(f"Always displayed: {context.is_always_displayed}")

✅ Context created with new API pattern!
Context ID: 6e9ef8a1-c6c0-462c-ad70-82db3b9b6c74
Always displayed: True


## 3. 🆕 Golden Examples - Simplified Creation & Bulk Operations

In [ ]:
# 🆕 NEW: Individual creation with keyword arguments
example1 = client.golden_examples.create(
    project_id=PROJECT_ID,
    user_query="What is our total revenue?",
    sql_query="SELECT SUM(quantity * unit_price) as total_revenue FROM orders WHERE status = 'completed';",
    description="Calculate total revenue from all completed orders",
    is_always_displayed=True
)

print("✅ Golden example created!")
print(f"Example ID: {example1.id}")

# 🆕 NEW: Bulk creation with list of dictionaries
examples_data = [
    {
        "user_query": "Who are our top 10 customers by total spending?",
        "sql_query": "SELECT c.customer_id, c.name, SUM(o.quantity * o.unit_price) as total_spent FROM customers c JOIN orders o ON c.customer_id = o.customer_id WHERE o.status = 'completed' GROUP BY c.customer_id, c.name ORDER BY total_spent DESC LIMIT 10;",
        "description": "Find top 10 customers by total spending"
    },
    {
        "user_query": "Show me monthly sales for this year",
        "sql_query": "SELECT DATE_TRUNC('month', order_date) as month, SUM(quantity * unit_price) as monthly_revenue FROM orders WHERE status = 'completed' AND EXTRACT(YEAR FROM order_date) = EXTRACT(YEAR FROM CURRENT_DATE) GROUP BY DATE_TRUNC('month', order_date) ORDER BY month;",
        "description": "Monthly sales trend for the current year"
    }
]

bulk_examples = client.golden_examples.bulk_create(
    project_id=PROJECT_ID,
    golden_examples=examples_data
)

print(f"✅ Bulk created {len(bulk_examples)} examples!")

✅ Golden example created with new API pattern!
Example ID: cf19a7ce-0ce3-4c0c-af18-dee729dafbd8
✅ Bulk created 2 examples with new API pattern!


## 4. 🆕 Schema Metadata - Simplified Creation & Bulk Operations

In [9]:
# 🆕 NEW: Individual creation with keyword arguments
customers_schema = client.schema_metadata.create(
    project_id=PROJECT_ID,
    name="customers_table",
    description="Customer information table",
    schema_data={
        "table": {
            "name": "customers",
            "columns": [
                {"name": "customer_id", "type": "INTEGER", "primary_key": True},
                {"name": "name", "type": "VARCHAR(100)", "nullable": False},
                {"name": "email", "type": "VARCHAR(255)", "unique": True},
                {"name": "registration_date", "type": "DATE"},
                {"name": "is_active", "type": "BOOLEAN", "default": True}
            ]
        }
    },
    is_always_displayed=True,
    validate=True
)

print("✅ Schema metadata created !")
print(f"Schema ID: {customers_schema.id}")

# 🆕 NEW: Bulk creation with list of dictionaries
schemas_data = [
    {
        "name": "orders_table",
        "description": "Order transactions table",
        "schema_data": {
            "table": {
                "name": "orders",
                "columns": [
                    {"name": "order_id", "type": "INTEGER", "primary_key": True},
                    {"name": "customer_id", "type": "INTEGER", "foreign_key": "customers.customer_id"},
                    {"name": "product_id", "type": "INTEGER", "foreign_key": "products.product_id"},
                    {"name": "quantity", "type": "INTEGER"},
                    {"name": "unit_price", "type": "DECIMAL(10,2)"},
                    {"name": "order_date", "type": "TIMESTAMP"},
                    {"name": "status", "type": "VARCHAR(20)", "default": "pending"}
                ]
            }
        }
    },
    {
        "name": "products_table",
        "description": "Product catalog table",
        "schema_data": {
            "table": {
                "name": "products",
                "columns": [
                    {"name": "product_id", "type": "INTEGER", "primary_key": True},
                    {"name": "name", "type": "VARCHAR(200)"},
                    {"name": "category", "type": "VARCHAR(50)"},
                    {"name": "price", "type": "DECIMAL(10,2)"},
                    {"name": "in_stock", "type": "BOOLEAN"}
                ]
            }
        }
    }
]

bulk_schemas = client.schema_metadata.bulk_create(
    project_id=PROJECT_ID,
    schema_metadata_list=schemas_data,
    validate=True
)

print(f"✅ Bulk created {len(bulk_schemas)} schemas !")

✅ Schema metadata created !
Schema ID: bf1d02d9-0433-41c4-a0ad-5e2530a70ee1
✅ Bulk created 2 schemas !


## 5. 🆕 Connectors - Simplified Creation with Smart Defaults

In [11]:
# 🆕 NEW: Direct keyword arguments with smart port defaults
# Using H2O AI Snowflake connector for demo data
connector = client.connectors.create(
    name="h2o-snowflake-connector",
    description="H2O AI Snowflake connector for Text2Everything demo data",
    db_type="snowflake",  # Smart default: port=443
    host=os.getenv("SNOWFLAKE_HOST"),
    username=os.getenv("SNOWFLAKE_USERNAME"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    config={
        "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
        "role": os.getenv("SNOWFLAKE_ROLE")
    }
)

# Alternative PostgreSQL example (commented out):
# connector = client.connectors.create(
#     name="E-commerce Database",
#     description="Production e-commerce PostgreSQL database",
#     db_type="postgres",  # Smart default: port=5432
#     host="localhost",
#     # port is optional - defaults to 5432 for postgres
#     username="ecommerce_user",
#     password="your_password",
#     database="ecommerce_db"
# )

print("✅ Connector created !")
print(f"Connector ID: {connector.id}")
print(f"Database Type: {connector.db_type}")
print(f"Host: {connector.host}:{connector.port}")
print(f"🆕 Smart default port: {connector.port} (auto-detected for {connector.db_type})")
print(f"Database: {connector.database}")
if hasattr(connector, 'config') and connector.config:
    print(f"Warehouse: {connector.config.get('warehouse', 'N/A')}")
    print(f"Role: {connector.config.get('role', 'N/A')}")

CONNECTOR_ID = connector.id

✅ Connector created !
Connector ID: 6f3b9450-733b-43c7-b10b-7dd9d2370a16
Database Type: snowflake
Host: H2OAIORG-H2O_AI_SNOWPARK_CONTAINERS.snowflakecomputing.com:443
🆕 Smart default port: 443 (auto-detected for snowflake)
Database: TEXT_2_EVERYTHING
Warehouse: TEXT_2_EVERYTHING_WH
Role: TEXT_2_EVERYTHING_ADMIN


## 6. 🆕 Chat Sessions - Simplified Creation

In [12]:
# 🆕 NEW: Direct keyword arguments
session = client.chat_sessions.create(
    project_id=PROJECT_ID,
    name="E-commerce Analytics Session",
    description="Interactive session for e-commerce data analysis"
)

print("✅ Chat session created !")
print(f"Session ID: {session.id}")
print(f"Session Name: {session.name}")

SESSION_ID = session.id

✅ Chat session created !
Session ID: 009ee8b1-b463-45ab-bd39-ade9a1a93c8d
Session Name: E-commerce Analytics Session


## 7. 🆕 Chat Methods - Simplified Text-to-SQL & Chat-to-Answer

In [ ]:
# 🆕 NEW: chat_to_sql with keyword arguments
sql_response = client.chat.chat_to_sql(
    project_id=PROJECT_ID,
    chat_session_id=SESSION_ID,
    query="What is our total revenue this year?",
    schema_metadata_id=customers_schema.id,
    contexts_limit=5,
    examples_limit=3
)

print("✅ SQL generated !")
print(f"Generated SQL: {sql_response.sql_query}")

# 🆕 NEW: chat_to_answer with keyword arguments (connector_id now required)
answer_response = client.chat.chat_to_answer(
    project_id=PROJECT_ID,
    chat_session_id=SESSION_ID,
    query="Version of the database",
    connector_id=CONNECTOR_ID,  # 🆕 Now required for better reliability
    schema_metadata_id=customers_schema.id,
    contexts_limit=5,
    examples_limit=3
)

print("✅ Answer generated !")
print(f"Answer: {answer_response.execution_result.result if hasattr(answer_response, 'execution_result') else 'N/A'}")
print(f"🆕 Required connector_id ensures reliable execution")

✅ SQL generated !
Generated SQL: SELECT SUM(quantity * unit_price) as total_revenue 
FROM orders 
WHERE status = 'completed' 
AND EXTRACT(YEAR FROM order_date) = EXTRACT(YEAR FROM CURRENT_DATE);
✅ Answer generated !
Answer: {'data': [['CURRENT_VERSION()'], ['9.25.1']], 'columns': ['CURRENT_VERSION()'], 'rowCount': 1}
🆕 Required connector_id ensures reliable execution


## 8. 🆕 Executions - Simplified SQL Execution

In [ ]:
# 🆕 NEW: execute_sql with keyword arguments
execution_result = client.executions.execute_sql(
    connector_id=CONNECTOR_ID,
    h2ogpte_session_id="session_123",
    sql_query="SELECT CURRENT_VERSION();"
)

print("✅ SQL executed !")
print(f"Execution ID: {execution_result.id}")
if execution_result.execution_result:
    print(f"Results: {execution_result.result}")

## 9. 🆕 Feedback - Simplified Feedback Creation

In [17]:
# 🆕 NEW: Direct keyword arguments for feedback
positive_feedback = client.feedback.create(
    project_id=PROJECT_ID,
    chat_message_id=sql_response.id,
    feedback="Perfect SQL query! Exactly what I needed.",
    is_positive=True,
    execution_id=execution_result.execution_id
)

print("✅ Positive feedback created !")
print(f"Feedback ID: {positive_feedback.id}")
print(f"Is Positive: {positive_feedback.is_positive}")

# 🆕 NEW: Helper methods also use new pattern
negative_feedback = client.feedback.create_negative(
    project_id=PROJECT_ID,
    chat_message_id=sql_response.id,
    feedback="Query could be improved with better filtering."
)

print("✅ Negative feedback created with helper method!")
print(f"Feedback ID: {negative_feedback.id}")

NameError: name 'execution_result' is not defined

## 10. 🆕 Update Operations - Simplified Updates

In [ ]:
# 🆕 NEW: Update operations with keyword arguments

# Update project
updated_project = client.projects.update(
    project_id=PROJECT_ID,
    name="E-commerce Analytics - Updated",
    description="Updated description with new insights"
)

print("✅ Project updated !")
print(f"New name: {updated_project.name}")

# Update connector
updated_connector = client.connectors.update(
    connector_id=CONNECTOR_ID,
    name="E-commerce Database - Production",
    description="Updated production database connector"
)

print("✅ Connector updated !")
print(f"New name: {updated_connector.name}")

# Update schema metadata
updated_schema = client.schema_metadata.update(
    schema_id=customers_schema.id,
    name="customers_table_v2",
    description="Updated customer information table with new fields"
)

print("✅ Schema metadata updated !")
print(f"New name: {updated_schema.name}")

In [ ]:
# Clean up
client.close()
print("✅ Client connection closed")
print("\n🎉 API Patterns Demo Complete!")